# Appointment Optimization
***
<br></br>
Please refer to MH4702_Project.pdf for complete problem description and result.

In this notebook we are simulating a clinic operation with one doctor as a server. This system can be categorized as a single server system with exponentialy distributed service rate, and fixed inter-arrival time (D/M/1).

required packages :
- pandas
- numpy
- bayes_opt (https://github.com/fmfn/BayesianOptimization)

In [1]:
import pandas as pd
import numpy as np
import math
import warnings
from functools import partial
from bayes_opt import BayesianOptimization

warnings.simplefilter("ignore")

np.random.seed(5)

rand_unif = np.random.uniform

## Simulation
***

Below are the descriptions of function parameters used in the simulation : <br>
- service_rate . Exponentialy distributed service rate of the doctor
- schedule_interval . Patient inter-arrival time. 
- opening_hours . Clinic opening hours. Clinic cannot accept new appointment beyond its opening hours.
- n_days . Simulation Duration in terms of days. Simulation will be generated on daily basis. <br>
- no_show_prob . Probability that a certain patient will not appear on his or her appointment time.


First customer arrive at time schedule_interval.

Some assumptions that are assumed to be true :
- Patient arrived on time or not at all.
- Patients are willing to wait for any time until he or she served by the doctor.


In [2]:
def simulate(service_rate = 3 , schedule_interval = 0.5 , opening_hours = 8 , n_days = 100 , no_show_prob = 0.0) :
    
    columns = [
        'Inter-arrival Time',
        'Arrival Time per Day',
        'Show up' ,
        'Service Starting Time',
        'Service Time',
        'Service Ending Time',
        'Waiting Time',
        'Sojourn Time',
        'Doctors Idle Time',
        'Number of People in the System',
        'Number of People Waiting',
    ]    
    
    cust_per_day = math.floor(opening_hours/schedule_interval)
    df_dict = {col : list() for col in columns}
    
    for i in range(n_days) :
        #### First Customer of the day
        inter_time = [schedule_interval]
        arrive_time = [schedule_interval]
        
        ###if random number bigger than no show probability mark as show up
        show_up = [1 if rand_unif() > no_show_prob else 0]
        
        service_s_time = [schedule_interval]
                    
        ##If show up do random roll for service time otherwise 0
        service_time = [-np.log(1-rand_unif())/service_rate if show_up[0] else 0]
        
        service_n_time = [service_s_time[0] + service_time[0]]
        wait_time = [service_s_time[0] - arrive_time[0]]
        
        sojourn_time = [service_n_time[0] - arrive_time[0]]
        doctors_idle = [0]
        n_people_sys = [1]
        n_people_wait = [max([n_people_sys[0] - 1 , 0])]
        
        for i in range(1, cust_per_day) :
            inter_time.append(schedule_interval)
            arrive_time.append(schedule_interval + arrive_time[i-1])
            
            if rand_unif() > no_show_prob :
                show_up.append(1)
            else :
                show_up.append(0)
                
            service_s_time.append(max([service_n_time[i-1] , arrive_time[i]])) 

            if show_up[i] :
                service_time.append(-np.log(1-rand_unif())/service_rate)
            else :
                service_time.append(0)
                
            service_n_time.append(service_s_time[i] + service_time[i])
            
            if show_up[i] :
                wait_time.append(service_s_time[i] - arrive_time[i])
            else :
                wait_time.append(0)
            sojourn_time.append(service_n_time[i] - arrive_time[i])
            
            doctors_idle.append(max([0 , arrive_time[i] - service_n_time[i-1]]))
            n_people_sys.append(sum((np.array(service_n_time[:i]) > arrive_time[i])*show_up[:i]) + show_up[i])
            n_people_wait.append(max([n_people_sys[i] - 1, 0]))
        
        df_dict['Inter-arrival Time'].extend(inter_time)
        df_dict['Arrival Time per Day'].extend(arrive_time)
        df_dict['Show up'].extend(show_up)
        df_dict['Service Starting Time'].extend(service_s_time)
        df_dict['Service Time'].extend(service_time)
        df_dict['Service Ending Time'].extend(service_n_time)
        df_dict['Waiting Time'].extend(wait_time)
        df_dict['Sojourn Time'].extend(sojourn_time)
        df_dict['Doctors Idle Time'].extend(doctors_idle)
        df_dict['Number of People in the System'].extend(n_people_sys)
        df_dict['Number of People Waiting'].extend(n_people_wait)
        
    return pd.DataFrame(df_dict)

### Simulation Sample


In [3]:
simulate(n_days = 1)

,Inter-arrival Time,Arrival Time per Day,Show up,Service Starting Time,Service Time,Service Ending Time,Waiting Time,Sojourn Time,Doctors Idle Time,Number of People in the System,Number of People Waiting
0,0.5,0.5,1,0.500000,0.681957,1.181957,0.000000,0.681957,0.000000,1,0
1,0.5,1.0,1,1.181957,0.836171,2.018128,0.181957,1.018128,0.000000,2,1
2,0.5,1.5,1,2.018128,0.315363,2.333491,0.518128,0.833491,0.000000,2,1
3,0.5,2.0,1,2.333491,0.243560,2.577051,0.333491,0.577051,0.000000,3,2
4,0.5,2.5,1,2.577051,0.069304,2.646355,0.077051,0.146355,0.000000,2,1
5,0.5,3.0,1,3.000000,0.447031,3.447031,0.000000,0.447031,0.353645,1,0
6,0.5,3.5,1,3.500000,0.057448,3.557448,0.000000,0.057448,0.052969,1,0
7,0.5,4.0,1,4.000000,0.106775,4.106775,0.000000,0.106775,0.442552,1,0
8,0.5,4.5,1,4.500000,0.117030,4.617030,0.000000,0.117030,0.393225,1,0
9,0.5,5.0,1,5.000000,0.289038,5.289038,0.000000,0.289038,0.382970,1,0


In order to be able to optimize the system we will define a scenario. Our optimization objective is profit maximization. 

We will split profit parts into income and cost :

<b>Income</b> : 
- For each patient that arrive at the clinic will get a fixed income of 600. <br>
- Patient who do not appear at his or her appointment will be regarded as sunk cost or income of 0.

<b>Cost</b> :
- Doctor daily hiring cost is assumed to be $1000 + 1800 \times n\_days \times \frac{opening\_hour}{8}$
- Doctor will be given overtime pay which amounts to $\frac{overtime\_duration}{5}$
- Operations cost will cover any other cost incurred by the clinic (utilities, rental, etc)

<b>Variables to optimize</b> :
- service_rate (default (unoptimized) = 3)
- schedule_interval (default = 0.5)
- opening_hours (optional) (default = 8)

To simplify the will not apply any constraints except limitations on our search space. 

For demonstration purposes we will show perform optimization with probability of not showing up as 0 and 0.2. I will also show comparison of the result before and after optimization.

Each simulate_scheme calls will perform simulation and returns the profit over n_days time window.


In [4]:
def income(show_up , fee = 500) :
#   Calculate the income based on the number of patient who show up.
    return fee * show_up.sum()

def operation_cost(over_time , over_time_multiplier ,  hiring_cost, op_hour , op_h_cost = 200) :
    over_time_cost = (over_time*over_time_multiplier).sum() 
    op_cost = 0
    for op in op_hour :
        op_cost += op_h_cost * op
    return over_time_cost + op_cost + hiring_cost

def simulate_scheme(service_rate , schedule_interval , opening_hours ,
                    op_h_cost = 200 , no_show_prob = 0.2 , fee = 400 , n_days = 60) :

    #### Simulate a scheme with given configuration of service rate schedule interval and opening hour.
    #### This simulation will return the profit for a given simulated period.
        
    #### op_h_cost : Hourly Operational Cost for the clinic
    #### no_show_prob : Number of patient not showing up , this patient are not chargeable.
    #### n_days : Number of days every simulation is evaluated. Higher n_days will probably yield more stable result.        
    #### hiring_cost : The cost for doctor.     
    #### *Assumption : Doctor with higher skill will serve faster hence able to handle more patients.
    
    #### Define Cost for doctor's skill, the doctor is paid daily cost is scaled according to his work hour, 
    #### using 8 as base working hour
    hiring_cost = (1000 + 1800*(service_rate))*(opening_hours/8)*n_days

    #### Assume the doctor is paid for 1/5 of his wage if he is doing overtime    
    over_time_multiplier = hiring_cost/(n_days*5.0)
    
    #### simulate the data for n_days 
    ref = simulate(service_rate , schedule_interval , opening_hours , n_days = n_days , no_show_prob = no_show_prob)
    
    #### How long the doctor does overtime. Values can be negative if he leave earlier 
    #### than his designated work hour ends
    ot_count = (ref[ref['Arrival Time per Day'] == ref['Arrival Time per Day'].max()]['Service Ending Time'] \
                - opening_hours - schedule_interval).values
    
    
    #### The function below implies that if the doctor finishes his service with the last patient 
    #### before his designated work hour ends, he can leave early. He is only entitled to overtime
    #### pays if he is past his work hour. This is that is often used in machine learning which is named
    #### rectified linear unit
    relu = lambda x : max(x,0)

    overtime = np.array(list(map(relu , ot_count)))
    op_hour = np.array(ot_count + opening_hours) 
    inc = income(ref['Show up'] , fee)
    cst = operation_cost(overtime , over_time_multiplier , hiring_cost , op_hour , op_h_cost)
    return inc - cst



In [5]:
optimize_op_hour = False

print('Single Server Optimization ...')    
base_parameter = {
    'service_rate' : 3 ,
    'schedule_interval' : 0.5
}

if optimize_op_hour :
    base_parameter['opening_hours'] = 8 

optimized = {}
print()

for no_show in [0,0.2] :

    print('With %.3f no show probability.' % no_show )
    
    fixed_params = {
    'op_h_cost' : 300 ,
    'no_show_prob' : no_show,
    'fee' : 600 ,
    'n_days' : 30 ,
    }
    
    if not optimize_op_hour :
        fixed_params['opening_hours'] = 8
    
    print()
    print('Base Parameters : ')
    for key in base_parameter.keys() :
        print('%s : %.3f' %(key ,base_parameter[key]))
    print()
    print('Fixed Parameters :')
    for key in fixed_params.keys() :
        print('%s : %.3f' % (key , fixed_params[key]))
    print()

   #### Simulate the scheme for 100 months to assess the clinic's profit.
    print('Before Optimization :')
    print()
    sim = simulate(**base_parameter , n_days = 30 , opening_hours = 8 , no_show_prob = no_show)
#    if no_show > 0 :
#        sim.to_csv('Unoptimized with Probability.csv' , index = False)
#    else :
#        sim.to_csv('Unoptimized without Probability.csv' , index = False)

    simulate_scheme_fixed = partial(simulate_scheme , **fixed_params)
    res = []
    for i in range(100) :
        res.append(simulate_scheme_fixed(**base_parameter))
    
    print('Monthly Profit Mean : %.3f' % np.mean(res))
    print('Monthly Profit Variance : %.3f' %  np.var(res))
    print('Monthly Profit std : %.3f' %  np.std(res))
    
    
    mean_service_time = sim[sim['Show up'] == 1]['Service Time'].mean()
    var_service_time = sim[sim['Show up'] == 1]['Service Time'].var()
    std_service_time = sim[sim['Show up'] == 1]['Service Time'].std()
    print()
    
    search_space = {'service_rate' : (1.0,5.0) ,
                    'schedule_interval' : (0.2,1.0)}
    
    if optimize_op_hour :
        search_space['opening_hours'] = (6.0,10.0)
    
    
    bo = BayesianOptimization(simulate_scheme_fixed , 
                              search_space,
                              random_state = 10, 
                              verbose = 0)
    
    bo.maximize(init_points = 5  , n_iter = 50)

    optimized[no_show] = bo.res['max']['max_params']

    print('After Optimization :')
    print()
    print('Best Parameters :')
    for key in optimized[no_show].keys() :
        print('%s : %.3f' %(key , optimized[no_show][key]))
    print()
    res = []

    for i in range(100) :
        res.append(simulate_scheme_fixed(**optimized[no_show]))
    
    print('Monthly Profit Mean : %.3f' %  np.mean(res))
    print('Monthly Profit Variance : %.3f' % np.var(res))
    print('Monthly Profit std : %.3f' % np.std(res))

    avg_idle = sim['Doctors Idle Time'].mean()
    var_idle = sim['Doctors Idle Time'].var()
    std_idle = sim['Doctors Idle Time'].std()
    avg_waiting = sim['Waiting Time'].mean()
    var_waiting = sim['Waiting Time'].var()
    std_waiting = sim['Waiting Time'].std()
    print()
    
    if optimize_op_hour :
        opening_hours = optimized[no_show]['opening_hours']
    else :
        opening_hours = fixed_params['opening_hours']
    
    #### Assess the queuing system performance in 30 days.
    print('Queueing performance :')
    sim = simulate(**optimized[no_show] , n_days = 30 , opening_hours = 8 , no_show_prob = no_show)
    maxval = sim[['Arrival Time per Day']].max().values[0]
    avg_overtime = np.mean([max(num , 0) for num in (sim[sim['Arrival Time per Day'] == maxval]['Service Ending Time'] - opening_hours - optimized[no_show]['schedule_interval']).values])
    var_overtime = np.var([max(num , 0) for num in (sim[sim['Arrival Time per Day'] == maxval]['Service Ending Time'] - opening_hours  - optimized[no_show]['schedule_interval']).values])
    std_overtime = np.std([max(num , 0) for num in (sim[sim['Arrival Time per Day'] == maxval]['Service Ending Time'] - opening_hours  - optimized[no_show]['schedule_interval']).values])
    
    print('Mean Service Time : %.3f' % mean_service_time)
    print('Variance Service Time : %.3f' % var_service_time)
    print('Standard Deviation Service Time :%.3f' % std_service_time)
    print()
    print('Average Waiting Time : %.3f' % avg_waiting)
    print('Variance Waiting Time : %.3f' % var_waiting)
    print('Standard Deviation Waiting Time : %.3f' % std_waiting)
    print()
    print('Average Idle Time : %.3f' % avg_idle)
    print('Variance Idle Time : %.3f'% var_idle)
    print('Standard Deviation Idle Time : %.3f'% std_idle)
    print()
    print('Average Overtime: %.3f ' % avg_overtime)
    print('Variance Overtime : %.3f' % var_overtime)
    print('Standard Deviation Overtime : %.3f' % std_overtime)
    print()
    print('Average Show up : %.3f' % sim['Show up'].mean())
    print('Variance Show up : %.3f' % sim['Show up'].var())
    print('Standard Deviation Show up : %.3f' % sim['Show up'].std())
    print('=======================================================================')
    print()
#    if no_show > 0 :
#        sim.to_csv('Optimized with No Show.csv' , index = False)
#    else :
#        sim.to_csv('Optimized without No Show.csv' , index = False)

Single Server Optimization ...

With 0.000 no show probability.

Base Parameters : 
service_rate : 3.000
schedule_interval : 0.500

Fixed Parameters :
op_h_cost : 300.000
no_show_prob : 0.000
fee : 600.000
n_days : 30.000
opening_hours : 8.000

Before Optimization :

Monthly Profit Mean : 14602.321
Monthly Profit Variance : 16815966.587
Monthly Profit std : 4100.728

After Optimization :

Best Parameters :
service_rate : 3.243
schedule_interval : 0.226

Monthly Profit Mean : 199963.320
Monthly Profit Variance : 244639219.326
Monthly Profit std : 15640.947

Queueing performance :
Mean Service Time : 0.351
Variance Service Time : 0.115
Standard Deviation Service Time :0.339

Average Waiting Time : 0.183
Variance Waiting Time : 0.125
Standard Deviation Waiting Time : 0.353

Average Idle Time : 0.156
Variance Idle Time : 0.030
Standard Deviation Idle Time : 0.174

Average Overtime: 2.737 
Variance Overtime : 1.756
Standard Deviation Overtime : 1.325

Average Show up : 1.000
Variance Show u